In [14]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import requests
import time
import math
import json

In [4]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
# options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)

In [5]:
# stealth(driver,
#     languages=["en-US", "en"],
#     vendor="Google Inc.",
#     platform="Win32",
#     webgl_vendor="Intel Inc.",
#     renderer="Intel Iris OpenGL Engine",
#     fix_hairline=True,
# )

In [10]:
driver.get("https://animepahe.ru/api?m=search&q=Classroom of the elites")

In [11]:
text =  driver.find_element(By.TAG_NAME, "pre")
jsonres = json.loads(text.get_attribute("innerHTML"))
print(jsonres)

{'total': 28, 'per_page': 8, 'current_page': 1, 'last_page': 4, 'from': 1, 'to': 8, 'data': [{'id': 2212, 'title': 'Classroom of the Elite', 'type': 'TV', 'episodes': 12, 'status': 'Finished Airing', 'season': 'Summer', 'year': 2017, 'score': 7.86, 'poster': 'https://i.animepahe.ru/posters/ec13a6ead81684e5105536f66ac0b205ca840e20e7ed7b17a003b7e58bde6251.jpg', 'session': '0abeaea1-718e-dfa4-99fd-9dd31e579802'}, {'id': 4810, 'title': 'Classroom of the Elite II', 'type': 'TV', 'episodes': 13, 'status': 'Finished Airing', 'season': 'Summer', 'year': 2022, 'score': 8.12, 'poster': 'https://i.animepahe.ru/posters/935d3af342283deca3325a5dc494169918eea63bed767ad7e15f02df7630813e.jpg', 'session': '417abac1-8014-c547-8e10-e0d0ae5cab4e'}, {'id': 5422, 'title': 'Classroom of the Elite III', 'type': 'TV', 'episodes': 13, 'status': 'Currently Airing', 'season': 'Winter', 'year': 2024, 'score': 8.15, 'poster': 'https://i.animepahe.ru/posters/068d48fcef22d608dc9b3109d1aaa2dc5bd5f67de56870b87b78747f73d

In [8]:
driver.get("https://google.com")

# input = driver.find_element(By.CLASS_NAME, "input-search")
# input.send_keys("One piece")

In [ ]:
# Get search results
# search_result_wrap = driver.find_element(By.CLASS_NAME, "search-results-wrap")

# while True:
#     try:
#         ul =  search_result_wrap.find_element(By.CLASS_NAME, "search-results")
#         li = ul.find_elements(By.TAG_NAME, "li")
#         for anime in li:
#             a = anime.find_element(By.TAG_NAME, "a")
#             result_title = a.find_element(By.CLASS_NAME, "result-title")
#             print (result_title.text.strip())
#         break
#     except Exception as e:
#         print (f"Couldn't find the results. Retrying in 5s: {e}")
#         time.sleep(5)

In [ ]:

response = requests.get("https://animepahe.ru/api?m=search&q=Classroom of the elites")
jsonres = None

if response.status_code == 200:
    # Print the response content (usually in JSON format for APIs)
    jsonres = response.json()
    print(jsonres)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")

In [12]:
driver.get(f"https://animepahe.ru/anime/{jsonres['data'][0]['session']}")

In [17]:
response = requests.get(f"https://animepahe.ru/api?m=release&id={jsonres['data'][0]['session']}&sort=episode_asc&page=1")
anime_details = response.text

anime_details

'<!doctype html><html><head><title>DDoS-Guard</title><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1"/><link rel="stylesheet" href="/.well-known/ddos-guard/js-challenge/index.css"><script defer="defer" src="/.well-known/ddos-guard/js-challenge/view.js"></script><script defer="defer" src="/.well-known/ddos-guard/js-challenge/index.js"></script><script src="https://check.ddos-guard.net/check.js"></script></head><body><div class="container"><div class="logo"><img id="img-logo" src="" alt="logo"/></div><div id="title"></div><div id="description"></div><div id="link-ddg"><a href="" target="_blank" id="link"></a></div><div class="lds"><img id="img-loading" src="" alt="loading"/></div></div></body></html>'

In [ ]:
# getting the episodes
def get_episode_link(episode, total):
    page = math.ceil(episode/30)
    response = requests.get(f"https://animepahe.ru/api?m=release&id={jsonres['data'][0]['session']}&sort=episode_asc&page={page}").json()
    position = (episode - (page - 1) * 30) - 1
    return f"https://animepahe.ru/play/{jsonres['data'][0]['session']}/{response['data'][position]['session']}"
        
link = get_episode_link(1, anime_details['total'])
link

In [ ]:

requests.get(
    "https://animepahe.ru/play/1f0c2e22-a432-cda9-7ac8-e6cbc7852a93/a1f3c490883a2f81cd39f1c915b8f0487d1827687d15531263ac3f600f331a77",
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15'
    }
).text

In [ ]:
driver.get(link)

In [ ]:
# Get quality lists
qualities = []

for a in driver.find_elements(By.CSS_SELECTOR, "#pickDownload .dropdown-item"):
    quality_text = a.get_attribute("innerText").split(" ")
    sub_dub = 'Dubbed' if quality_text[-1] == 'eng' else 'Subbed'
    quality = quality_text[2]
    size = quality_text[3][1:-1]
    quality = {
        "name": f"{quality}\t\t{size}\t\t{sub_dub}", #a.get_attribute("innerHTML").split(" ")[2],
        "link": a.get_attribute("href")
    }
    qualities.append(quality)

qualities

In [ ]:
qualities[0]['link']

In [ ]:
driver.get(qualities[0]['link'])

In [ ]:
# Get the continue button
link_to_download = ""

while True:
    try:
        a = driver.find_element(By.XPATH, "//a[text()='Continue']")
        link_to_download = a.get_attribute("href")
        break
    except:
        print("Couldn't find button, waiting 7s")
        time.sleep(7)
 
link_to_download

In [ ]:
driver.get(link_to_download)

In [ ]:
# Find button and click it
link = driver.find_element(By.CSS_SELECTOR, ".main .download form").get_attribute("action")
token = driver.find_element(By.CSS_SELECTOR, ".main .download form input").get_attribute("value")
print("Link:", link)
print("Token:", token)


In [ ]:
# cookies = {}
# for cookie in driver.get_cookies():
#     cookies[cookie['name']] = cookie['value']
# cookies

cookies = '; '.join([
    f"{cookie['name']}={cookie['value']}"
    for cookie in driver.get_cookies()
])
cookies

In [ ]:
data = {
    "_token": token
}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0',
  'Referer': 'https://kwik.cx/f/r6gCLSqIe9Nh',
  'Origin': 'https://kwik.cx',
  'Host': 'kwik.cx',
  'Cookie': cookies
}

In [ ]:

data = {
    "_token": token
}
response = requests.post(link, data=data, headers=headers, stream=True)
file_name = response.headers.get('Content-Disposition').split('=')[1]
print(file_name)

response.raise_for_status()

total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte

progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

with open(file_name, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)

progress_bar.close()